In [23]:
import pandas as pd
import os

!pip install tqdm
import tqdm

!pip install jdatetime
import jdatetime

<h1>.xlsx &rarr; .csv</h1>

In [51]:
# Convert xlsx files to

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
year = "1401"
path = "/Users/mammedbrk/Uni/032/regression/traffic-data"

for month in months:
    month_path = os.path.join(path, year, month)
    for f in tqdm.tqdm(os.listdir(month_path)):
        filename = os.path.join(month_path, f)
        if not os.path.isfile(filename) or not f.endswith(".xlsx"):
            continue
        df = pd.read_excel(filename, header=1)
        df.to_csv(filename[:-5] + ".csv", encoding='utf-8', index=0, header=1)


100%|██████████| 2686/2686 [00:31<00:00, 84.08it/s]


<h1>Average estimated traffic count for each road</h1>

In [39]:
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
year = "1401"
path = "/Users/mammedbrk/Uni/032/regression/traffic-data"


df_map = {}

for month in months:

    month_path = os.path.join(path, year, month)
    for f in tqdm.tqdm(os.listdir(month_path)):
        if not f.endswith(".csv"):
            continue

        df = pd.read_csv(os.path.join(month_path, f), header=0)

        code = df['کد محور'][0]
        if not code in df_map:
            df_map[code] = df
        else:
            df_map[code] = pd.concat([df_map[code], df], ignore_index=True)

100%|██████████| 32/32 [00:00<00:00, 218952.25it/s]


In [ ]:
df = pd.DataFrame([
    {
        'code': code,
        'mean': df['تعداد برآورد شده'].mean().astype(int),
        'std': df['تعداد برآورد شده'].std().astype(int)
    }
    for code, df in df_map.items()
])


df = df.sort_values('code')
df.to_csv(f"/Users/mammedbrk/Uni/032/regression/traffic-data/mean_count_{year}.csv", index=0, header=1)

/var/folders/3w/dqcbdkz93vq_ylwghvn_hmkr0000gn/T/ipykernel_59082/342450445.py:5: RuntimeWarning: invalid value encountered in cast
  'std': df['تعداد برآورد شده'].std().astype(int)


<h1>Extract Nowruz data</h1>

In [52]:
year = "1402"
pre_year = str(int(year) - 1)
path = "/Users/mammedbrk/Uni/032/regression/traffic-data"

esfand = os.path.join(path, pre_year, '12')
farvardin = os.path.join(path, year, '01')

In [53]:
df_mean = pd.read_csv(f"/Users/mammedbrk/Uni/032/regression/traffic-data/mean_count_1402.csv", header=0)

In [54]:
def clean_df(df):
    df.rename(columns={
        'کد محور': 'code',
        'نام محور': 'name', 
        'زمان شروع': 'date',
        'مدت زمان کارکرد (دقیقه)': 'duration',
        'تعداد کل وسیله نقلیه': 'total_count',
        'تعداد وسیله نقلیه کلاس 1': 'car_count',
        'سرعت متوسط': 'avg_speed',
        'تعداد برآورد شده': 'total_est'
    }, inplace=True)

    df.loc[:, 'car_est'] = (
        df['total_est'] * df['car_count'] // df['total_count']
    )

    stats = df_mean[df_mean['code'] == df['code'][0]].iloc[0]
    df.loc[:, 'total_est_normal'] = (
        (df['total_est'] - stats['mean']) / stats['std']
    )

    df = df[['code', 'name', 'date', 'duration', 'total_count', 'car_count', 'total_est', 'car_est', 'avg_speed', 'total_est_normal']]

    df.loc[:, 'date'] = df['date'].str[:-9]
    df.loc[:, 'date'] = df['date'].str.replace('/', '-')
    df.loc[:, 'date'] = df['date'].apply(lambda x: jdatetime.datetime.strptime(x, "%Y-%m-%d").date())

    return df

In [55]:
def get_slice(st_day, fn_day, df):
    # Convert comparison date
    st_date = jdatetime.datetime.strptime(f"{pre_year}-12-{st_day}", "%Y-%m-%d").date()
    fn_date = jdatetime.datetime.strptime(f"{year}-01-{fn_day}", "%Y-%m-%d").date()

    df = df[(df['date'] >= st_date) & (df['date'] <= fn_date)]
    return df

In [ ]:
for f in os.listdir(esfand):
    if not os.path.isfile(os.path.join(esfand, f)) or not f.endswith('.csv'):
        continue
    if not os.path.exists(os.path.join(farvardin, f)):
        print(f"{f} not exists.")
        continue

    df_esfand = pd.read_csv(os.path.join(esfand, f))
    df_farvardin = pd.read_csv(os.path.join(farvardin, f))
    df_nowruz = pd.concat([df_esfand, df_farvardin], ignore_index=True)

    df_nowruz = clean_df(df_nowruz)

    df_nowruz = get_slice('10', '20', df_nowruz)

    df_nowruz.to_csv(f"/Users/mammedbrk/Uni/032/regression/traffic-data/nowruz_02/{f}", index=False, header=True)

In [ ]:
df_road = pd.DataFrame(columns=['code', 'name'])
nowruz_path = "/Users/mammedbrk/Uni/032/regression/traffic-data/nowruz_03"

for f in os.listdir(nowruz_path):
    if not os.path.isfile(os.path.join(nowruz_path, f)) or not f.endswith('.csv'):
        continue
    
    df = pd.read_csv(os.path.join(nowruz_path, f))
    df = df[['code', 'name']]
    df_road = pd.concat([df_road, df.iloc[[0]]], ignore_index=True)

df_road.head()

,code,name
0,673703,خدابنده - بيجار (خورخوره)
1,933702,مهريز - انار (مهريز)
2,873454,آزادراه گرمسار - قم (گرمسار) (متناظر توزين)
3,214209,جاده قديم مورچه‌خورت - نطنز (سه راهي مورچه خورت)
4,833702,مهران - دهلران


In [100]:
df_road = df_road.sort_values('code')
df_road.head()

,code,name
1995,113201,آزادراه قم - گرمسار (چرمشهر)
1814,113202,شهريار-کرج (هفت جوي)
1671,113203,آزادراه تهران - قم (مجتمع ياس)
1497,113206,آزادراه تهران - پرديس (عوارضي پرديس)
1641,113207,آزادراه تهران - پرديس (بومهن)


In [101]:
df_road.to_csv("/Users/mammedbrk/Uni/032/regression/road_data_total.csv", index=False, header=True)

In [128]:
import pandas as pd
import re

def parse_road_name(name):
    # Initialize components
    road_type = None
    origin = None
    destination = None
    place = None
    
    # Common patterns to match
    patterns = [
        # Pattern 1: آزادراه قم - گرمسار (چرمشهر)
        (r'^(آزادراه|کمربندي|محور|بزرگراه|جاده قديم|جاده|راهجديد|راه دسترسي|راه|کنارگذر|عوارضي)\s+([^()]+?)\s*-\s*([^()]+?)(?:\s*\((.*)\))?$', 
         lambda m: (m.group(1).strip(), m.group(2).strip(), m.group(3).strip(), m.group(4))),
        
        # Pattern 2: شهريار-کرج (هفت جوي)
        (r'^([^()]+?)\s*-\s*([^()]+?)(?:\s*\((.*)\))?$', 
         lambda m: (None, m.group(1).strip(), m.group(2).strip(), m.group(3))),
        
        # Pattern 3: آزادراه تهران- شمال (قطعه اول - محدوده عوارضي تهران)
        (r'^(آزادراه|جاده|کمربندي)\s+([^()]+?)\s*-\s*([^()]+?)\s*\((.*)\)$', 
         lambda m: (m.group(1).strip(), m.group(2).strip(), m.group(3).strip(), m.group(4))),
        
        # Pattern 4: جاده قديم تهران، قم (حسن‌آباد - نعلبندان)
        (r'^(جاده قديم)\s+([^()،]+?)،\s*([^()]+?)\s*\((.*)\)$', 
         lambda m: (m.group(1).strip(), m.group(2).strip(), m.group(3).strip(), m.group(4))),

        # Pattern 5: 
        (r'^(کمربندي)\s+([^()]+?)\s*\((.*)\)$', 
         lambda m: (m.group(1).strip(), m.group(2).strip(), None, m.group(3).strip())),
    ]
    
    # Try each pattern in order
    for pattern, handler in patterns:
        match = re.match(pattern, name)
        if match:
            road_type, origin, destination, place = handler(match)
            if place:  # Clean up place if it exists
                place = place.strip()
            break
    
    # If no pattern matched, try to split by -
    if origin is None and '-' in name:
        parts = name.split('-', 1)
        origin = parts[0].strip()
        rest = parts[1].strip()
        
        # Check if rest has parentheses
        if '(' in rest and ')' in rest:
            place_start = rest.find('(')
            destination = rest[:place_start].strip()
            place = rest[place_start+1:rest.find(')')].strip()
        else:
            destination = rest
    
    # Clean up origin/destination if they have road type
    if origin and not road_type:
        for rt in ['آزادراه', 'جاده', 'کمربندي', 'محور', 'بزرگراه', 'جاده قديم']:
            if origin.startswith(rt):
                road_type = rt
                origin = origin[len(rt):].strip()
                break
    
    # Clean up components
    if road_type:
        road_type = road_type.replace('ي', 'ی').replace('ك', 'ک')
    if origin:
        origin = origin.replace('ي', 'ی').replace('ك', 'ک')
    if destination:
        destination = destination.replace('ي', 'ی').replace('ك', 'ک')
    if place:
        place = place.replace('ي', 'ی').replace('ك', 'ک')
    
    return road_type, origin, destination, place

def process_road_data(input_file, output_file):
    # Read the input data
    df = pd.read_csv(input_file)
    
    # Initialize new columns
    df['type'] = None
    df['origin'] = None
    df['destination'] = None
    df['place'] = None
    
    # Process each row
    for idx, row in df.iterrows():
        name = row['name']
        if pd.isna(name):
            continue
            
        road_type, origin, destination, place = parse_road_name(name)
        
        df.at[idx, 'type'] = road_type
        df.at[idx, 'origin'] = origin
        df.at[idx, 'destination'] = destination
        df.at[idx, 'place'] = place
    
    # Reorder columns
    df = df[['code', 'type', 'origin', 'destination', 'place']]
    
    # Save to output file
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
# Specify the input and output file paths
input_file = '/Users/mammedbrk/Uni/032/regression/road_data_total.csv'
output_file = '/Users/mammedbrk/Uni/032/regression/processed_road_data.csv'

# Process the CSV file
process_road_data(input_file, output_file)

print(f"Processed data has been saved to {output_file}")


Processed data has been saved to /Users/mammedbrk/Uni/032/regression/processed_road_data.csv


In [220]:
import pandas as pd
import jdatetime
from typing import List

def find_missing_jdates(df: pd.DataFrame, 
                       date_column: str = 'date',
                       freq: str = 'D') -> List[jdatetime.date]:
    """
    Find missing Jalali (Persian) dates in a DataFrame.
    
    Args:
        df: Input DataFrame
        date_column: Name of column containing Jdatetime dates
        freq: Frequency to check ('D' for daily, 'M' for monthly, etc.)
    
    Returns:
        List of missing Jdatetime dates
    """
    # Convert to Jdatetime if not already
    if not isinstance(df[date_column].iloc[0], jdatetime.date):
        df.loc[:, 'date'] = df['date'].apply(lambda x: jdatetime.datetime.strptime(x, "%Y-%m-%d").date())
    
    
    # Create complete date range
    min_date = jdatetime.datetime.strptime("1401-12-10", "%Y-%m-%d").date()
    max_date = jdatetime.datetime.strptime("1402-01-20", "%Y-%m-%d").date()
    
    if freq == 'D':
        complete_range = [min_date + jdatetime.timedelta(days=x) 
                         for x in range((max_date - min_date).days + 1)]
    elif freq == 'M':
        # For monthly frequency
        complete_range = []
        current = min_date.replace(day=1)
        while current <= max_date:
            complete_range.append(current)
            if current.month == 12:
                current = jdatetime.date(current.year + 1, 1, 1)
            else:
                current = jdatetime.date(current.year, current.month + 1, 1)
    
    # Find missing dates
    existing_dates = set(df[date_column])
    missing_dates = [d for d in complete_range if d not in existing_dates]
    
    print(existing_dates)
    dict = {}
    for d in complete_range:
        if d not in existing_dates:
            print(d)
            df_1, df_2 = None, None
            if d > min_date:
                df_1 = df[df['date'] == d - jdatetime.timedelta(days=1)].iloc[0]
            if d < max_date:
                df_2 = df[df['date'] == d + jdatetime.timedelta(days=1)].iloc[0]

            df_1 = df_2 if df_1 is None else df_1
            df_2 = df_1 if df_2 is None else df_2

            for col in df.columns:
                if pd.api.types.is_numeric_dtype(df[col]):
                    dict[col] = (df_1[col] + df_2[col]) / 2
                dict['date'] = d
                dict['name'] = df['name']

    
    return pd.DataFrame([dict])

In [221]:
nowruz_path = "/Users/mammedbrk/Uni/032/regression/traffic-data/nowruz_02"

for f in os.listdir(nowruz_path):
    if not os.path.isfile(os.path.join(nowruz_path, f)) or not f.endswith('.csv'):
        continue
    
    df = pd.read_csv(os.path.join(nowruz_path, f))
    l = len(df)
    if l < 40 and l > 38:
        print(f"{f}")
        missing_dates = find_missing_jdates(df)
        row = find_missing_jdates(df)
        df = pd.concat([df, row], ignore_index=True).sort_values('date').reset_index(drop=True)
        df.to_csv(os.path.join(nowruz_path, f), index=False, header=True)


163213.csv
{jdatetime.date(1402, 1, 19), jdatetime.date(1401, 12, 15), jdatetime.date(1401, 12, 13), jdatetime.date(1401, 12, 24), jdatetime.date(1401, 12, 20), jdatetime.date(1402, 1, 14), jdatetime.date(1402, 1, 1), jdatetime.date(1402, 1, 4), jdatetime.date(1401, 12, 18), jdatetime.date(1401, 12, 23), jdatetime.date(1402, 1, 16), jdatetime.date(1402, 1, 17), jdatetime.date(1402, 1, 18), jdatetime.date(1401, 12, 14), jdatetime.date(1402, 1, 3), jdatetime.date(1402, 1, 12), jdatetime.date(1402, 1, 7), jdatetime.date(1401, 12, 12), jdatetime.date(1402, 1, 13), jdatetime.date(1401, 12, 16), jdatetime.date(1401, 12, 25), jdatetime.date(1401, 12, 19), jdatetime.date(1402, 1, 5), jdatetime.date(1402, 1, 9), jdatetime.date(1401, 12, 28), jdatetime.date(1402, 1, 11), jdatetime.date(1401, 12, 29), jdatetime.date(1402, 1, 20), jdatetime.date(1401, 12, 27), jdatetime.date(1402, 1, 6), jdatetime.date(1402, 1, 8), jdatetime.date(1401, 12, 21), jdatetime.date(1402, 1, 15), jdatetime.date(1401, 12,